In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import datetime as dt

df_crash = pd.read_csv("C:\\Users\Rimmen\crash_data.csv", low_memory = False)
df_crash["CRASH DATE"] = df_crash["CRASH DATE"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d") )
df_crash["CRASH TIME"] = df_crash["CRASH TIME"].apply(lambda x: datetime.strptime(x, "%H:%M:%S") )

In [146]:
#ML
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [2]:
df_crash

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2019-09-23,1900-01-01 12:30:00,BROOKLYN,NaN,40.641327,-74.007090,POINT (-74.00709 40.641327),52 STREET,NaN,NaN,...,Unspecified,Unspecified,NaN,NaN,4211039,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN
1,2019-09-24,1900-01-01 15:35:00,BROOKLYN,NaN,40.660675,-74.001380,POINT (-74.00138 40.660675),3 AVENUE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4212543,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN
2,2019-10-03,1900-01-01 12:00:00,QUEENS,11369.0,40.767467,-73.881870,POINT (-73.88187 40.767467),88 STREET,23 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4217205,Sedan,NaN,NaN,NaN,NaN
3,2019-10-10,1900-01-01 20:05:00,BROOKLYN,11221.0,40.691300,-73.945500,POINT (-73.9455 40.6913),TOMPKINS AVENUE,KOSCIUSZKO STREET,NaN,...,Unspecified,NaN,NaN,NaN,4221415,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN
4,2019-09-23,1900-01-01 13:40:00,MANHATTAN,10037.0,40.812813,-73.941810,POINT (-73.94181 40.812813),LENOX AVENUE,WEST 133 STREET,NaN,...,Unspecified,NaN,NaN,NaN,4212429,Station Wagon/Sport Utility Vehicle,E-Sco,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531320,2013-01-08,1900-01-01 22:01:00,QUEENS,11365.0,40.732922,-73.800367,POINT (-73.8003671 40.7329223),69 AVENUE,169 STREET,NaN,...,Unspecified,NaN,NaN,NaN,234690,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
1531321,2013-01-05,1900-01-01 00:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Unspecified,NaN,NaN,NaN,2997723,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
1531322,2013-01-13,1900-01-01 20:30:00,BROOKLYN,11230.0,40.612550,-73.962859,POINT (-73.962859 40.6125498),CONEY ISLAND AVENUE,AVENUE O,NaN,...,Unspecified,NaN,NaN,NaN,131491,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE,NaN,NaN,NaN
1531323,2013-01-10,1900-01-01 08:45:00,QUEENS,11369.0,40.762172,-73.870400,POINT (-73.8704003 40.7621717),ASTORIA BOULEVARD,99 STREET,NaN,...,Unspecified,NaN,NaN,NaN,281078,TAXI,PASSENGER VEHICLE,NaN,NaN,NaN


In [15]:
ML = df_crash[['CRASH DATE','CRASH TIME','BOROUGH','CONTRIBUTING FACTOR VEHICLE 1']]

In [8]:
ML['CONTRIBUTING FACTOR VEHICLE 1'].unique()

array(['Other Vehicular', 'Passing Too Closely',
       'Driver Inattention/Distraction', 'Aggressive Driving/Road Rage',
       'Passing or Lane Usage Improper', 'Unspecified', 'Unsafe Speed',
       'Unsafe Lane Changing', 'Alcohol Involvement',
       'Following Too Closely', 'Failure to Yield Right-of-Way',
       'Backing Unsafely', 'Passenger Distraction', 'Driver Inexperience',
       'Pedestrian/Bicyclist/Other Pedestrian Error/Confusion',
       'Turning Improperly', 'Reaction to Uninvolved Vehicle', nan,
       'Brakes Defective', 'Accelerator Defective', 'Eating or Drinking',
       'Traffic Control Disregarded', 'View Obstructed/Limited',
       'Failure to Keep Right', 'Illnes', 'Oversized Vehicle',
       'Drugs (illegal)', 'Steering Failure', 'Glare', 'Fell Asleep',
       'Prescription Medication', 'Physical Disability',
       'Lost Consciousness', 'Driverless/Runaway Vehicle',
       'Tow Hitch Defective', 'Animals Action', 'Pavement Slippery',
       'Fatigued/Drowsy

In [126]:
ML2 = ML.loc[ML['CONTRIBUTING FACTOR VEHICLE 1'].isin(['Alcohol Involvement','Pavement Slippery'])].copy()

In [127]:
ML2['CRASH DATE'] = ML2['CRASH DATE'].dt.year
ML2['CRASH TIME'] = ML2['CRASH TIME'].dt.hour

In [128]:
ML2['25mph'] = 0
ML2.loc[(ML2['CRASH DATE'] != 2013) & (ML2['CRASH DATE'] != 2014),'25mph'] = 1 

In [129]:
ML2.rename(columns={'CRASH DATE':'year','CRASH TIME':'hour','CONTRIBUTING FACTOR VEHICLE 1':'reason','BOROUGH':'borough',},inplace=True)

In [130]:
ML2 = ML2.loc[ML2.borough.notna()].copy()

In [131]:
ML2.drop(['year'],axis=1,inplace=True)

In [132]:
ML2

,hour,borough,reason,25mph
16,2,BROOKLYN,Alcohol Involvement,1
226,23,BROOKLYN,Alcohol Involvement,1
535,20,BROOKLYN,Alcohol Involvement,1
564,19,BROOKLYN,Alcohol Involvement,1
613,15,BRONX,Pavement Slippery,1
...,...,...,...,...
1530759,1,QUEENS,Pavement Slippery,0
1530802,1,STATEN ISLAND,Alcohol Involvement,0
1530947,23,BRONX,Pavement Slippery,0
1531173,18,BROOKLYN,Alcohol Involvement,0


In [231]:
crash = pd.concat((ML2,pd.get_dummies(ML2.borough)),1)

In [232]:
crash.drop(['borough'],axis=1,inplace=True)

In [233]:
crash.columns = map(str.lower, crash.columns)

In [234]:
crash

,hour,reason,25mph,bronx,brooklyn,manhattan,queens,staten island
16,2,Alcohol Involvement,1,0,1,0,0,0
226,23,Alcohol Involvement,1,0,1,0,0,0
535,20,Alcohol Involvement,1,0,1,0,0,0
564,19,Alcohol Involvement,1,0,1,0,0,0
613,15,Pavement Slippery,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...
1530759,1,Pavement Slippery,0,0,0,0,1,0
1530802,1,Alcohol Involvement,0,0,0,0,0,1
1530947,23,Pavement Slippery,0,1,0,0,0,0
1531173,18,Alcohol Involvement,0,0,1,0,0,0


In [147]:
#Splitting data into features (X) and labels (y)
X = crash.drop(['reason'],axis=1)
y = crash['reason']

#Randomly choosing 20% of the data as our test data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [150]:
#fitting our Random Forest Classifier
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train,y_train);

In [151]:
pred_rfc=rfc.predict(X_test)
print('Confusion matrix:')
print(confusion_matrix(y_test,pred_rfc))

Confusion matrix:
[[1847  809]
 [ 877 1805]]


In [152]:
print('Classification Report:')
print(classification_report(y_test,pred_rfc))

Classification Report:
                     precision    recall  f1-score   support

Alcohol Involvement       0.68      0.70      0.69      2656
  Pavement Slippery       0.69      0.67      0.68      2682

           accuracy                           0.68      5338
          macro avg       0.68      0.68      0.68      5338
       weighted avg       0.68      0.68      0.68      5338



In [155]:
#Fraction of 'VEHICLE THEFT' labels in our testing data
print("Fraction of 'Pavement Slippery' in our testdata: ", y_test.value_counts()[0]/y_test.value_counts().sum())

#Fraction of 'VEHICLE THEFT' labels in our training data
print("Fraction of 'Pavement Slippery' in our trainingdata: ",y_train.value_counts()[0]/y_train.value_counts().sum())

Fraction of 'Pavement Slippery' in our testdata:  0.5024353690520794
Fraction of 'Pavement Slippery' in our trainingdata:  0.501100754133683


In [154]:
y_test.value_counts()

Pavement Slippery      2682
Alcohol Involvement    2656
Name: reason, dtype: int64

In [156]:
#Dropping 25mph
X2_train= X_train.copy()
X2_test = X_test.copy()
y2_train = y_train.copy()
y2_test = y_test.copy()
X2_train = X2_train.drop(['25mph'],axis=1)
X2_test = X2_test.drop(['25mph'],axis=1)

In [158]:
#fitting our Random Forest Classifier and presenting our results
rfc2 = RandomForestClassifier(n_estimators=100)
rfc2.fit(X2_train,y2_train)
pred_rfc2=rfc2.predict(X2_test)
print('Classification Report:')
print(classification_report(y2_test,pred_rfc2))
print('Confusion matrix:')
print(confusion_matrix(y2_test,pred_rfc2))

Classification Report:
                     precision    recall  f1-score   support

Alcohol Involvement       0.68      0.70      0.69      2656
  Pavement Slippery       0.69      0.67      0.68      2682

           accuracy                           0.68      5338
          macro avg       0.69      0.68      0.68      5338
       weighted avg       0.69      0.68      0.68      5338

Confusion matrix:
[[1858  798]
 [ 884 1798]]


In [159]:
#Dropping hour
X3_train= X_train.copy()
X3_test = X_test.copy()
y3_train = y_train.copy()
y3_test = y_test.copy()
X3_train = X3_train.drop(['hour'],axis=1)
X3_test = X3_test.drop(['hour'],axis=1)
#fitting our Random Forest Classifier and presenting our results
rfc3 = RandomForestClassifier(n_estimators=100)
rfc3.fit(X3_train,y3_train)
pred_rfc3=rfc3.predict(X3_test)
print('Classification Report:')
print(classification_report(y3_test,pred_rfc3))
print('Confusion matrix:')
print(confusion_matrix(y3_test,pred_rfc3))

Classification Report:
                     precision    recall  f1-score   support

Alcohol Involvement       0.53      0.54      0.53      2656
  Pavement Slippery       0.54      0.53      0.53      2682

           accuracy                           0.53      5338
          macro avg       0.53      0.53      0.53      5338
       weighted avg       0.53      0.53      0.53      5338

Confusion matrix:
[[1428 1228]
 [1264 1418]]


In [162]:
ML2['hour'].unique()

array([ 2, 23, 20, 19, 15, 14, 18,  4,  3,  7,  9,  1,  5, 16,  6, 21,  0,
        8, 13, 17, 22, 11, 12, 10], dtype=int64)

In [163]:
ML2

,hour,borough,reason,25mph
16,2,BROOKLYN,Alcohol Involvement,1
226,23,BROOKLYN,Alcohol Involvement,1
535,20,BROOKLYN,Alcohol Involvement,1
564,19,BROOKLYN,Alcohol Involvement,1
613,15,BRONX,Pavement Slippery,1
...,...,...,...,...
1530759,1,QUEENS,Pavement Slippery,0
1530802,1,STATEN ISLAND,Alcohol Involvement,0
1530947,23,BRONX,Pavement Slippery,0
1531173,18,BROOKLYN,Alcohol Involvement,0


In [174]:
combs = ML2.groupby(['hour','borough','25mph']).size().reset_index()

In [175]:
combs

,hour,borough,25mph,0
0,0,BRONX,0,38
1,0,BRONX,1,212
2,0,BROOKLYN,0,82
3,0,BROOKLYN,1,406
4,0,MANHATTAN,0,72
...,...,...,...,...
235,23,MANHATTAN,1,148
236,23,QUEENS,0,91
237,23,QUEENS,1,318
238,23,STATEN ISLAND,0,12


In [ ]:
print(rfc.predict([[32,0,0,0,0,1,0,0,0,0,0]]))

In [177]:
crash

,hour,reason,25mph,bronx,brooklyn,manhattan,queens,staten island
16,2,Alcohol Involvement,1,0,1,0,0,0
226,23,Alcohol Involvement,1,0,1,0,0,0
535,20,Alcohol Involvement,1,0,1,0,0,0
564,19,Alcohol Involvement,1,0,1,0,0,0
613,15,Pavement Slippery,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...
1530759,1,Pavement Slippery,0,0,0,0,1,0
1530802,1,Alcohol Involvement,0,0,0,0,0,1
1530947,23,Pavement Slippery,0,1,0,0,0,0
1531173,18,Alcohol Involvement,0,0,1,0,0,0


In [213]:
crash2 = crash.drop(['reason'],axis=1)

In [214]:
crash2.drop_duplicates(inplace=True)

In [225]:
crash.columns.tolist

<bound method IndexOpsMixin.tolist of Index(['hour', 'reason', '25mph', 'bronx', 'brooklyn', 'manhattan', 'queens',
       'staten island'],
      dtype='object')>

In [227]:
cols = ['hour', '25mph', 'bronx', 'brooklyn', 'manhattan', 'queens',
       'staten island','reason']

In [235]:
crash2 = crash

In [236]:
crash2=crash2[cols].copy()

In [237]:
crash2

,hour,25mph,bronx,brooklyn,manhattan,queens,staten island,reason
16,2,1,0,1,0,0,0,Alcohol Involvement
226,23,1,0,1,0,0,0,Alcohol Involvement
535,20,1,0,1,0,0,0,Alcohol Involvement
564,19,1,0,1,0,0,0,Alcohol Involvement
613,15,1,1,0,0,0,0,Pavement Slippery
...,...,...,...,...,...,...,...,...
1530759,1,0,0,0,0,1,0,Pavement Slippery
1530802,1,0,0,0,0,0,1,Alcohol Involvement
1530947,23,0,1,0,0,0,0,Pavement Slippery
1531173,18,0,0,1,0,0,0,Alcohol Involvement


In [238]:
crash2['predictions'] = rfc.predict(crash2.iloc[:,:-1].values)

In [239]:
crash2

,hour,25mph,bronx,brooklyn,manhattan,queens,staten island,reason,predictions
16,2,1,0,1,0,0,0,Alcohol Involvement,Alcohol Involvement
226,23,1,0,1,0,0,0,Alcohol Involvement,Alcohol Involvement
535,20,1,0,1,0,0,0,Alcohol Involvement,Alcohol Involvement
564,19,1,0,1,0,0,0,Alcohol Involvement,Alcohol Involvement
613,15,1,1,0,0,0,0,Pavement Slippery,Pavement Slippery
...,...,...,...,...,...,...,...,...,...
1530759,1,0,0,0,0,1,0,Pavement Slippery,Alcohol Involvement
1530802,1,0,0,0,0,0,1,Alcohol Involvement,Alcohol Involvement
1530947,23,0,1,0,0,0,0,Pavement Slippery,Alcohol Involvement
1531173,18,0,0,1,0,0,0,Alcohol Involvement,Pavement Slippery
